#Model Building

In [2]:
import os
import pprint
import numpy as np

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [3]:
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.21

In [4]:
# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /databricks/driver/aml_config/config.json
Workspace name: azuremlservice
Azure region: westeurope
Resource group: azuremlserviceresourcegroup

In [5]:
#get the train and test datasets
train_data_path = "AdultCensusIncomeTrain"
test_data_path = "AdultCensusIncomeTest"

train = spark.read.parquet(train_data_path)
test = spark.read.parquet(test_data_path)

print("train: ({}, {})".format(train.count(), len(train.columns)))
print("test: ({}, {})".format(test.count(), len(test.columns)))

train.printSchema()

train: (25967, 15)
test: (6594, 15)
root
-- age: integer (nullable = true)
-- race: string (nullable = true)
-- fnlwgt: integer (nullable = true)
-- sex: string (nullable = true)
-- occupation: string (nullable = true)
-- capital_loss: integer (nullable = true)
-- relationship: string (nullable = true)
-- hours_per_week: integer (nullable = true)
-- education_num: integer (nullable = true)
-- native_country: string (nullable = true)
-- education: string (nullable = true)
-- marital_status: string (nullable = true)
-- workclass: string (nullable = true)
-- capital_gain: integer (nullable = true)
-- income: string (nullable = true)

#Define Model

In [7]:
label = "income"
dtypes = dict(train.dtypes)
dtypes.pop(label)

si_xvars = []
ohe_xvars = []
featureCols = []
for idx,key in enumerate(dtypes):
    if dtypes[key] == "string":
        featureCol = "-".join([key, "encoded"])
        featureCols.append(featureCol)
        
        tmpCol = "-".join([key, "tmp"])
        # string-index and one-hot encode the string column
        #https://spark.apache.org/docs/2.3.0/api/java/org/apache/spark/ml/feature/StringIndexer.html
        #handleInvalid: Param for how to handle invalid data (unseen labels or NULL values). 
        #Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), 
        #or 'keep' (put invalid data in a special additional bucket, at index numLabels). Default: "error"
        si_xvars.append(StringIndexer(inputCol=key, outputCol=tmpCol, handleInvalid="skip"))
        ohe_xvars.append(OneHotEncoder(inputCol=tmpCol, outputCol=featureCol))
    else:
        featureCols.append(key)

# string-index the label column into a column named "label"
si_label = StringIndexer(inputCol=label, outputCol='label')

# assemble the encoded feature columns in to a column named "features"
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

In [8]:
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import numpy as np
import os
import shutil

model_name = "AdultCensus_runHistory.mml"
model_dbfs = os.path.join("/dbfs", model_name)
run_history_name = 'spark-ml-notebook'

# start a training run by defining an experiment
myexperiment = Experiment(ws, "Ignite_AI_Talk")
root_run = myexperiment.start_logging()

# Regularization Rates - 
regs = [0.0001, 0.001, 0.01, 0.1]

# try a bunch of regularization rate in a Logistic Regression model
for reg in regs:
    print("Regularization rate: {}".format(reg))
    # create a bunch of child runs
    with root_run.child_run("reg-" + str(reg)) as run:
        # create a new Logistic Regression model.
        lr = LogisticRegression(regParam=reg)
        
        # put together the pipeline
        pipe = Pipeline(stages=[*si_xvars, *ohe_xvars, si_label, assembler, lr])

        # train the model
        model_p = pipe.fit(train)
        
        # make prediction
        pred = model_p.transform(test)
        
        # evaluate. note only 2 metrics are supported out of the box by Spark ML.
        bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
        au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
        au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

        print("Area under ROC: {}".format(au_roc))
        print("Area Under PR: {}".format(au_prc))
      
        # log reg, au_roc, au_prc and feature names in run history
        run.log("reg", reg)
        run.log("au_roc", au_roc)
        run.log("au_prc", au_prc)
        run.log_list("columns", train.columns)

        # save model
        model_p.write().overwrite().save(model_name)
        
        # upload the serialized model into run history record
        mdl, ext = model_name.split(".")
        model_zip = mdl + ".zip"
        shutil.make_archive(mdl, 'zip', model_dbfs)
        run.upload_file("outputs/" + model_name, model_zip)        
        #run.upload_file("outputs/" + model_name, path_or_stream = model_dbfs) #cannot deal with folders

        # now delete the serialized model from local folder since it is already uploaded to run history 
        shutil.rmtree(model_dbfs)
        os.remove(model_zip)
        
# Declare run completed
root_run.complete()
root_run_id = root_run.id
print ("run id:", root_run.id)

Regularization rate: 0.0001
Area under ROC: 0.9065598035775412
Area Under PR: 0.7672969207724432
Regularization rate: 0.001
Area under ROC: 0.9060472775317929
Area Under PR: 0.7656341943684336
Regularization rate: 0.01
Area under ROC: 0.9032445938782964
Area Under PR: 0.7531967474759237
Regularization rate: 0.1
Area under ROC: 0.8948774138109985
Area Under PR: 0.72951523414279
run id: 18d8b7a1-a8cf-455f-b71b-63e3aea607ca

In [9]:
#Load all run metrics from run history into a dictionary object.
child_runs = {}

for r in root_run.get_children():
    child_runs[r.id] = r

In [10]:
metrics = root_run.get_metrics(recursive=True)
best_run_id = max(metrics, key = lambda k: metrics[k]['au_roc'])
best_run = child_runs[best_run_id]
print('Best run is:', best_run_id)
print('Metrics:', metrics[best_run_id]['au_roc'], metrics[best_run_id]['reg'])

Best run is: e3bec785-aa6a-4cc0-ad7e-213588e81686
Metrics: 0.9065598035775412 0.0001

In [11]:
#Download the model from the best run to a local folder
best_model_file_name = "best_model.zip"
best_run.download_file(name = 'outputs/' + model_name, output_file_path = best_model_file_name)

#Model Evaluation

In [13]:
##unzip the model to dbfs (as load() seems to require that) and load it.
if os.path.isfile(model_dbfs) or os.path.isdir(model_dbfs):
    shutil.rmtree(model_dbfs)
shutil.unpack_archive(best_model_file_name, model_dbfs)

model_p_best = PipelineModel.load(model_name)

In [14]:
# make prediction
pred = model_p_best.transform(test)
output = pred[['hours_per_week','age','marital_status','income','prediction']]
display(output.limit(8))

hours_per_week,age,marital_status,income,prediction
20,17,Never-married,<=50K,0.0
24,17,Never-married,<=50K,0.0
40,17,Never-married,<=50K,0.0
30,17,Never-married,<=50K,0.0
20,17,Never-married,<=50K,0.0
40,17,Never-married,<=50K,0.0
12,17,Never-married,<=50K,0.0
10,17,Never-married,<=50K,0.0


In [15]:
# evaluate. note only 2 metrics are supported out of the box by Spark ML.
bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

print("Area under ROC: {}".format(au_roc))
print("Area Under PR: {}".format(au_prc))

Area under ROC: 0.9065598035775412
Area Under PR: 0.7672969207724432

#Model Persistence

In [17]:
##NOTE: by default the model is saved to and loaded from /dbfs/ instead of cwd!
model_p_best.write().overwrite().save(model_name)
print("saved model to {}".format(model_dbfs))

saved model to /dbfs/AdultCensus_runHistory.mml

In [18]:
%sh

ls -la /dbfs/AdultCensus_runHistory.mml/*

/dbfs/AdultCensus_runHistory.mml/metadata:
total 0
drwxr-xr-x 1 root root 0 Apr 2 17:00 .
drwxr-xr-x 1 root root 0 Apr 2 17:00 ..
-rw-r--r-- 1 root root 736 Apr 2 17:00 part-00000
-rw-r--r-- 1 root root 0 Apr 2 17:00 _SUCCESS

/dbfs/AdultCensus_runHistory.mml/stages:
total 0
drwxr-xr-x 1 root root 0 Apr 2 17:00 .
drwxr-xr-x 1 root root 0 Apr 2 17:00 ..
drwxr-xr-x 1 root root 0 Apr 2 17:00 00_StringIndexer_196ec0765f4c
drwxr-xr-x 1 root root 0 Apr 2 17:00 01_StringIndexer_2837873189b1
drwxr-xr-x 1 root root 0 Apr 2 17:00 02_StringIndexer_f3af5359e4b8
drwxr-xr-x 1 root root 0 Apr 2 17:00 03_StringIndexer_e38c02ba5274
drwxr-xr-x 1 root root 0 Apr 2 17:00 04_StringIndexer_f82826711f0a
drwxr-xr-x 1 root root 0 Apr 2 17:00 05_StringIndexer_93ec1ffdc4ac
drwxr-xr-x 1 root root 0 Apr 2 17:00 06_StringIndexer_6afb81996426
drwxr-xr-x 1 root root 0 Apr 2 17:00 07_StringIndexer_10dd82818507
drwxr-xr-x 1 root root 0 Apr 2 17:00 08_OneHotEncoder_9f98df3dbeca
drwxr-xr-x 1 root root 0 Apr 2 17:00 09_OneHotEncoder_75f4a4160ee0
drwxr-xr-x 1 root root 0 Apr 2 17:00 10_OneHotEncoder_cf1a571c1ace
drwxr-xr-x 1 root root 0 Apr 2 17:00 11_OneHotEncoder_1187a6e749ff
drwxr-xr-x 1 root root 0 Apr 2 17:00 12_OneHotEncoder_3de98e0c5b35
drwxr-xr-x 1 root root 0 Apr 2 17:00 13_OneHotEncoder_f95cf1308b16
drwxr-xr-x 1 root root 0 Apr 2 17:00 14_OneHotEncoder_2b12e1064029
drwxr-xr-x 1 root root 0 Apr 2 17:00 15_OneHotEncoder_ab9f57d003b2
drwxr-xr-x 1 root root 0 Apr 2 17:00 16_StringIndexer_84ea2f09648b
drwxr-xr-x 1 root root 0 Apr 2 17:01 17_VectorAssembler_0acbc4af9f2d
drwxr-xr-x 1 root root 0 Apr 2 17:01 18_LogisticRegression_29e94df829fb

In [19]:
dbutils.notebook.exit("success")

success